In [4]:
import xgi
import igraph as ig
import time
import networkx as nx

In [2]:
H_enron = xgi.load_xgi_data("email-enron")
H_enron_cleaned = H_enron.cleanup(
    multiedges=False, singletons=False, isolates=False, relabel=True, in_place=False
)

In [ ]:
start_time = time.time()

H_nx = xgi.convert.to_graph(H_enron)

middle_time = time.time()

H_ig = xgi.convert.to_igraph(H_enron)

end_time = time.time()

print(f"Время выполнения nx: {(middle_time-start_time):.6f} секунд")
print(f"Время выполнения ig: {(end_time-middle_time):.6f} секунд")

Время выполнения nx: 0.064836 секунд
Время выполнения ig: 0.041606 секунд


In [5]:
start_time = time.time()

cent_nx = nx.pagerank(H_nx)

middle_time = time.time()

cent_ig = H_ig.pagerank()

end_time = time.time()

print(f"Время выполнения nx: {(middle_time-start_time):.6f} секунд")
print(f"Посчитанная центральность nx: {cent_nx}")
print(f"Время выполнения ig: {(end_time-middle_time):.6f} секунд")
print(f"Посчитанная центральность ig: {cent_ig}")

Время выполнения nx: 0.017001 секунд
Посчитанная центральность nx: {'4': 0.008716154071140611, '1': 0.012163470499910517, '117': 0.0076197338296881065, '129': 0.007471375365005757, '51': 0.010924228158413218, '41': 0.012703639074126083, '65': 0.00954057053810252, '107': 0.009325077668512361, '122': 0.003042709863927914, '29': 0.008853027407650172, '133': 0.004197199882501676, '62': 0.005592149891662153, '71': 0.004325284181686286, '97': 0.012054386023844845, '63': 0.014117886533092844, '120': 0.00863311885856544, '98': 0.007076520352514017, '74': 0.006045602469771838, '23': 0.00950147601989895, '147': 0.011081502027622826, '20': 0.008136803885410552, '57': 0.007763015132701736, '125': 0.008644779622575184, '116': 0.008129317358049087, '108': 0.009233819891238516, '44': 0.006303850885861543, '79': 0.006625611311876394, '8': 0.01369989829429635, '112': 0.008763084156455405, '142': 0.008083752877890567, '136': 0.01265399529391787, '132': 0.006160116939720925, '52': 0.008472764283676637, '